In [16]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [17]:
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

In [23]:
mesh_stern = 'mobley_test/1112_tetrachloroethane/surf_d02_stern'
mesh_diel = 'mobley_test/1112_tetrachloroethane/surf_d02'
atom_vdw = 'mobley_test/1112_tetrachloroethane/1112_tetrachloroethane.vdw'
atom_xyzr = numpy.loadtxt('mobley_test/1112_tetrachloroethane/1112_tetrachloroethane.xyzr')



qe = 1.60217662e-19
Na = 6.0221409e23

atom_pos = atom_data[:,:3]
atom_r   = atom_data[:,3]
N_atom = len(atom_r)
atom_eps = numpy.zeros(N_atom)

i=0
for line in file(atom_vdw):
    line = line.split()
    atom_eps[i] = float(line[-1])
    i += 1
    
rho_w = 0.0336 # 1/angs3 number density of water at standard conditions
water_r = 1.7682 # angs
water_eps = 0.1520 # kcal/mol

grid = generate_grid(mesh_diel)

N_panel = grid.leaf_view.entity_count(0)
vertices = grid.leaf_view.vertices 
triangles = grid.leaf_view.elements
elements = list(grid.leaf_view.entity_iterator(0))

area = numpy.zeros(N_panel)
center = numpy.zeros((N_panel,3))
normal = numpy.zeros((N_panel,3))
for i in range(N_panel):
    area[i] = elements[i].geometry.volume
    center[i,:] = numpy.average(elements[i].geometry.corners[:],axis=1)
    v1 = elements[i].geometry.corners[:,1] - elements[i].geometry.corners[:,0]
    v2 = elements[i].geometry.corners[:,2] - elements[i].geometry.corners[:,0]
    normal[i,:] = numpy.cross(v1,v2)/(2*area[i]) 

integral_i = numpy.zeros(N_atom)
for i in range(N_atom):
    r_local = center - atom_pos[i,:]
    r_norm = numpy.sqrt(numpy.sum(r_local**2, axis=1))
    rdn = numpy.sum(r_local*normal, axis=1)
    epsilon = numpy.sqrt(water_eps*atom_eps[i])
    
    A = epsilon*(water_r + atom_r[i])**12
    B = 2*epsilon*(water_r + atom_r[i])**6
    print A, B
        
    r_large = numpy.where(r_norm>0.4*atom_r[i])[0]
    print 'Triangles out: %i'%(len(center)-len(r_large))
    
    integral_i[i] = numpy.sum((-A/(9*r_norm[r_large]**12) + B/(3*r_norm[r_large]**6)) * rdn[r_large] * area[r_large])
    
energy = rho_w*numpy.sum(integral_i)
print energy
print integral_i

1247453.59731 944.186781794
Triangles out: 0
798296.045632 604.22333614
Triangles out: 0
1598416.35689 1132.78455426
Triangles out: 0
1598416.35689 1132.78455426
Triangles out: 0
1598416.35689 1132.78455426
Triangles out: 0
1598416.35689 1132.78455426
Triangles out: 0
53474.6584855 102.220927117
Triangles out: 0
53474.6584855 102.220927117
Triangles out: 0
-613.989006689
[ -5.90743611e+02   3.46115552e+00  -3.84794719e+03  -3.35243590e+03
  -3.38492194e+03  -3.38301338e+03  -1.88407740e+03  -1.83380407e+03]


In [ ]:
(1.244+3.177+0.105)/3

In [ ]:
1.73310520523/0.14104208421

In [ ]:
print elements[0].geometry